# Ray Paralelization of Making Mels Spectrograms using

Key takeaways:

1. Library Ray seems to be outperforming in this setup.

2. It is possible that one can obtain similar performance with some care.  

3. Because of low overhead you do not have to think of big batches with ray.  

Note: one has to be careful though as I found no good wait to check on processes finish except for the dirty method I use below of waiting for 60 seconds of no writing to disk.  Works here.

# General Imports

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.interactive(False)
matplotlib.use('Agg')

# find all of the files in the directory
import os
import gc
import sys

import warnings
warnings.filterwarnings('ignore')

import sys
from tqdm.notebook import tqdm
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

printfile=None
#open('log.txt','w')

# Launching Ray


In [ ]:
!pip install ray
import ray
ray.shutdown()
ray.init(num_cpus=4,include_webui=False)

# Various Paths and Training

In [ ]:
#gpath='C:/Users/karab/Documents/GitHub/birdsong-recognition/train_audio/'

ipath='../input/birdsong-recognition/train_audio/'
opath= "./"
opath= "./data"

birds = next(os.walk(ipath))[1]
ff=[] # list of all files
count=0;

birds = np.sort(birds)
birds = [b for b in birds if b[0]<='h']

for i, bird in enumerate(birds):
    for _, _, _files in os.walk(ipath+bird):
        count = count+1;
        print("{:03} {:7} {:3} files\t".format(i,bird,len(_files)),end='\t' if (i+1)%5 else '\n',file=printfile)
        ff.append(_files)

# Core

In [ ]:
def saveMel(signal, directory,sr):
    gc.enable()
    # spectrogram parameters
    N_FFT = 1024         # 
    HOP_SIZE = 1024      #  
    N_MELS = 128         # h   
    WIN_SIZE = 1024      # 
    WINDOW_TYPE = 'hann' # 
    FEATURE = 'mel'      # 
    FMIN = 1400

    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    S = librosa.feature.melspectrogram(y=signal, sr=sr,
                                        n_fft=N_FFT,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, # higher limit ##high-pass filter freq.
                                        fmax=sr/2) # AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=FMIN) #power = S**2
    
    fig.savefig(directory)
    plt.ioff()
    #plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:
size = {'desired': 5,  # [seconds]
        'minimum': 4,  # [seconds]
        'stride': 0,  # [seconds]
        'name': 5  # [number of letters]
        }  # stride should not be bigger than desired length

@ray.remote
def worker(infile,outfile,verbose=True):
    warnings.filterwarnings('ignore')
    try:    
        signal, sr = librosa.load(infile,sr=None)  # sr = sampling rate
        step = (size['desired']-size['stride']) * sr # length of step between two cuts in seconds
        count = 0;
        if VERBOSITY and (sr>0): print('\n{:03} {:12} {:>6.0f}    {} sample'.format(j, file, len(signal),sr))#, end='' if (j+1)%5 else '\n')
        for start, end in (zip(range(0, len(signal), step), range(size['desired']*sr, len(signal), step))):
            count = count+1
            os.makedirs(os.path.dirname(outfile), exist_ok=True)
            saveMel(signal[start:end], outfile.format(count),sr) #scratch: if end-start > size['minimum']*sr else signal[-step:]
            os.rename(outfile.format(count),outfile.format(count))
            if VERBOSITY: print(count,end=' ')
        #print(infile,'completed',file=printfile)
    except:
        print('Loading of ',infile, 'failed',file=printfile)
        #return infile


TEST=False
VERBOSITY=0


if TEST: birds = birds[:5]
tqdm=lambda x,y,z:x
results = []
print('Number of birds: ', len(birds))
for i, bird in tqdm(enumerate(birds),'Main',len(birds)):#enumerate(birds):# 
    for _, _, _files in os.walk(ipath+bird):
        infiles,outfiles=[],[]
        if TEST: _files = _files[:5]
        for j,file in tqdm(enumerate(_files),bird,len(_files)):#enumerate(_files):#
            infile = os.path.join(ipath, bird, file)
            outfile = os.path.join(opath, bird, file).replace('.mp3', '_{}.png')
            results.append(worker.remote(infile,outfile))

# Dirty pool

In [ ]:
import time
import glob
count = len(glob.glob('/kaggle/working/data/*/*'))
time.sleep(60)
while count<len(glob.glob('/kaggle/working/data/*/*')):
    count=len(glob.glob('/kaggle/working/data/*/*'))
    time.sleep(60)
    print(count)


# PNGs -> tar.gz

Somehow saving png files just in directory leads to not even being able to load this notebook in viewer afterwards or them not being seen so

In [ ]:
import tarfile
import os
import shutil
import glob

file_name = "data.tar.gz"
os.chdir('/kaggle/working/')
tar = tarfile.open(file_name, "w:gz")
for f in glob.glob('./data/*/*'):
    tar.add(f)
    os.remove(f)
shutil.rmtree('./data')
tar.close()